In [2]:
import requests 
import yaml
import json
import datetime as dt
import time
import os
import sys
import random
from azure.storage.blob import BlobServiceClient, BlobType
from email_validator import validate_email, EmailNotValidError
from itertools import chain

# pathing 
PARENT_DIR = os.path.abspath(os.path.pardir)
module_path = os.path.abspath(os.path.join(PARENT_DIR)) # for other functions
if module_path not in sys.path:
  sys.path.append(module_path)
from funcs.utils import log_azure, request, load_json, clean_field_text, has_valid_email, load_config
from funcs.funcs import combine_qa_keys, get_sm_survey_responses, process_sm_responses, post_cos

DATA_DIR = os.path.join(PARENT_DIR, "data")

# creds
with open(os.path.join(PARENT_DIR, "creds", "api-key.yaml"), "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = data['sm']['real']
# CareerOneStop Survey 
COS_DATA = data['cos']

IndentationError: expected an indented block after 'if' statement on line 354 (funcs.py, line 356)


---

##### **Main Functions** 

* `get_qa_key()` - GET (or load cached copy) of question/answer key from SM or COS 

<br>

* `combine_qa_keys()` - Combine the SM and COS question/answer keys into one combined key/translation map between the APIs
    - Generates a refreshed map if a change is detected in the SM survey or the COS survey. 
        - The COS survey should not change at all. 
        - The survey monkey questions which match to the COS survey questions should not change (they are intended to just be a port).

<br>

* `get_sm_responses()` - GET SM survey responses 

<br>

* `process_sm_responses()` - filter and process new SM survey responses from get_sm_responses()
    - Checks against DB for already processed responses 
    - Checks for unexpected question ids vs. the combined Q/A key.
        - Attempts to refresh the combined Q/A key if any unexpected question ids are found.
    - Adds matching information from the combined Q/A key to the survey responses   
    - Loads new responses into database (into 'processing' table) until they are finished

<br>

* `post_cos()` 
    - Creates COS JSON request objects from each SM survey response object 
        - If a SM survey response is missing an answer for a skills-survey question, it fills the corresponding question in the COS object with an answer of "Beginner"
    -  POSTS each request object to the COS Skills Matcher API 
    - Stores the COS response alongside the original SM survey response, updating the database 
    
<br>

* `compose_email()` - Compose an email from a COS response object.

<br>

* `send_email()` - Send email if respondent provided valid email address. 

<br>

* `export_csv()` - Export data from database in csv/excel format for Social Contract analysis purposes      

In [3]:
sm_survey_responses = get_sm_survey_responses()

INFO: GET {'https://api.surveymonkey.com/v3/surveys/409913146/responses/bulk'} -- {200} -- 1.56 -- {datetime.datetime(2023, 10, 16, 11, 11, 23, 828991)}


In [15]:
sm_survey_responses['data'][0]['pages'][0]['questions']

[{'id': '156451072',
  'answers': [{'other_id': '1149785635',
    'text': 'I am working on it (Alex)'}]},
 {'id': '156451074', 'answers': [{'choice_id': '1149785637'}]},
 {'id': '156450994', 'answers': [{'tag_data': [], 'text': '19711'}]},
 {'id': '156450995',
  'answers': [{'row_id': '1149785273', 'tag_data': [], 'text': '04/07/1997'}]},
 {'id': '156451075', 'answers': [{'choice_id': '1149785640'}]},
 {'id': '156450996', 'answers': [{'choice_id': '1149785282'}]}]

In [6]:
processed_responses = process_sm_responses(sm_survey_responses)
processed_responses

/Users/alexsocarras/tech_impact/career-onestop/funcs/utils.py:95: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


[{'response_id': '114437777551',
  'collector_id': '428261927',
  'questions': [{'question_id': {'sm': '156451072'},
    'page_number': 1,
    'question_number': {'sm': 1},
    'question_family': 'single_choice',
    'question_text': {'sm': 'How did you learn about the survey?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1149785635'},
      'text': {'sm': 'I am working on it (Alex)'}}]},
   {'question_id': {'sm': '156451074'},
    'page_number': 1,
    'question_number': {'sm': 2},
    'question_family': 'single_choice',
    'question_text': {'sm': 'Are you taking this survey online or in person?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1149785637'}, 'text': {'sm': 'Online'}}]},
   {'question_id': {'sm': '156450994'},
    'page_number': 1,
    'question_number': {'sm': 3},
    'question_family': 'open_ended',
    'question_text': {'sm': 'What zip code do you currently live in?'},
    'question_type': 'non-skills-matche

In [16]:
combined_keys = combine_qa_keys()
for q in list(combined_keys['skills-matcher'].values()):
    display(q)

/Users/alexsocarras/tech_impact/career-onestop/funcs/utils.py:95: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


{'question_id': {'sm': '156451029', 'cos': '2.C.1.a'},
 'page_number': 21,
 'question_number': {'sm': 42, 'cos': 1},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How much do you know about business planning and leadership?',
  'cos': 'How much do you know about business planning and leadership?'},
 'answers': [{'id': {'sm': '1149785390', 'cos': 1.534},
   'text': {'sm': 'Beginner (Complete a timesheet)',
    'cos': 'Complete a timesheet'}},
  {'id': {'sm': '1149785391', 'cos': 2.4145},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785392', 'cos': 3.295},
   'text': {'sm': 'Skilled (Monitor project progress to complete it on time)',
    'cos': 'Monitor project progress to complete it on time'}},
  {'id': {'sm': '1149785393', 'cos': 4.1755},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785394', 'cos': 5.056},
   'text': {'sm': 'Expert (Manage a $10m company)',
    'cos': 'Manage a $10m company'}}]}

{'question_id': {'sm': '156451030', 'cos': '2.C.4.d'},
 'page_number': 21,
 'question_number': {'sm': 43, 'cos': 2},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How much do you know about plant, animal and cell functions?',
  'cos': 'How much do you know about plant, animal and cell functions?'},
 'answers': [{'id': {'sm': '1149785395', 'cos': 1.372},
   'text': {'sm': 'Beginner (Care for a pet)', 'cos': 'Care for a pet'}},
  {'id': {'sm': '1149785396', 'cos': 2.401},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785397', 'cos': 3.43},
   'text': {'sm': 'Skilled (Investigate effects of pollution on plants)',
    'cos': 'Investigate effects of pollution on plants'}},
  {'id': {'sm': '1149785398', 'cos': 4.459},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785399', 'cos': 5.488},
   'text': {'sm': 'Expert (Identify a new virus)',
    'cos': 'Identify a new virus'}}]}

{'question_id': {'sm': '156451031', 'cos': '1.A.3.c.3'},
 'page_number': 21,
 'question_number': {'sm': 44, 'cos': 3},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How well can you coordinate moving your arms, legs, and torso together?',
  'cos': 'How well can you coordinate moving your arms, legs, and torso together?'},
 'answers': [{'id': {'sm': '1149785400', 'cos': 1.176},
   'text': {'sm': 'Beginner (Get in and out of a truck)',
    'cos': 'Get in and out of a truck'}},
  {'id': {'sm': '1149785401', 'cos': 2.058},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785402', 'cos': 2.94},
   'text': {'sm': 'Skilled (Swim one pool length, or play a ball sport)',
    'cos': 'Swim one pool  length, or play a ball sport'}},
  {'id': {'sm': '1149785403', 'cos': 3.822},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785404', 'cos': 4.704},
   'text': {'sm': 'Expert (Perform ballet choreography)',
    'cos': 'Perform ballet choreography'}}]}

{'question_id': {'sm': '156451032', 'cos': '2.C.3.d'},
 'page_number': 21,
 'question_number': {'sm': 45, 'cos': 4},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How much do you know about construction materials, methods, and tools?',
  'cos': 'How much do you know about construction materials, methods, and tools?'},
 'answers': [{'id': {'sm': '1149785405', 'cos': 1.24},
   'text': {'sm': 'Beginner (Hang a picture)', 'cos': 'Hang a picture'}},
  {'id': {'sm': '1149785406', 'cos': 2.17},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785407', 'cos': 3.1},
   'text': {'sm': 'Skilled (Fix a plumbing leak in ceiling)',
    'cos': 'Fix a plumbing leak in ceiling'}},
  {'id': {'sm': '1149785408', 'cos': 4.03},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785409', 'cos': 4.96},
   'text': {'sm': 'Expert (Build a high-rise)', 'cos': 'Build a high-rise'}}]}

...

{'question_id': {'sm': '156451064', 'cos': '2.C.9.a'},
 'page_number': 21,
 'question_number': {'sm': 77, 'cos': 36},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How well do you understand telecom systems?',
  'cos': 'How well do you understand telecom systems?'},
 'answers': [{'id': {'sm': '1149785566', 'cos': 1.208},
   'text': {'sm': 'Beginner (Use a business phone)',
    'cos': 'Use a business phone'}},
  {'id': {'sm': '1149785567', 'cos': 2.114},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785568', 'cos': 3.02},
   'text': {'sm': 'Skilled (Install a satellite TV dish)',
    'cos': 'Install a satellite TV dish'}},
  {'id': {'sm': '1149785569', 'cos': 3.926},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785570', 'cos': 4.832},
   'text': {'sm': 'Expert (Develop a worldwide telecom network)',
    'cos': 'Develop a worldwide telecom network'}}]}

{'question_id': {'sm': '156451065', 'cos': '2.C.5.b'},
 'page_number': 21,
 'question_number': {'sm': 78, 'cos': 37},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How well do you know the principles and methods used in counseling?',
  'cos': 'How well do you know the principles and methods used in counseling?'},
 'answers': [{'id': {'sm': '1149785571', 'cos': 1.384},
   'text': {'sm': 'Beginner (Offer sympathy to a stranger)',
    'cos': 'Offer sympathy to a stranger'}},
  {'id': {'sm': '1149785572', 'cos': 2.422},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785577', 'cos': 3.46},
   'text': {'sm': 'Skilled (Provide job counseling to the unemployed)',
    'cos': 'Provide job counseling to the unemployed'}},
  {'id': {'sm': '1149785578', 'cos': 4.498},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785579', 'cos': 5.536},
   'text': {'sm': 'Expert (Supervise experienced therapists)',
    'cos': 'Supervise experienced therapists'}}]}

{'question_id': {'sm': '156451066', 'cos': '2.B.5.a'},
 'page_number': 21,
 'question_number': {'sm': 79, 'cos': 38},
 'question_family': 'single_choice',
 'question_text': {'sm': "How well do you manage your and others' time?",
  'cos': "How well do you manage your and others' time?"},
 'answers': [{'id': {'sm': '1149785580', 'cos': 2.054},
   'text': {'sm': 'Beginner (Keep a monthly appointment calendar)',
    'cos': 'Keep a monthly appointment calendar'}},
  {'id': {'sm': '1149785581', 'cos': 2.5595},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785582', 'cos': 3.065},
   'text': {'sm': 'Skilled (Allocate staff time to projects for the coming week)',
    'cos': 'Allocate staff time to projects for the coming week'}},
  {'id': {'sm': '1149785583', 'cos': 3.5705},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785584', 'cos': 4.076},
   'text': {'sm': 'Expert (Allocate scientists’ time to multiple multi-stage projects)',
    'cos': 'Allocate scientists

{'question_id': {'sm': '156451067', 'cos': '2.B.3.k'},
 'page_number': 21,
 'question_number': {'sm': 80, 'cos': 39},
 'question_family': 'single_choice',
 'question_text': {'sm': 'How well can you identify and fix problems in machines or technology?',
  'cos': 'How well can you identify and fix problems in machines or technology?'},
 'answers': [{'id': {'sm': '1149785585', 'cos': 1.05},
   'text': {'sm': 'Beginner (Identify source of leak on outside of machine)',
    'cos': 'Identify source of leak on outside of machine'}},
  {'id': {'sm': '1149785586', 'cos': 1.8375},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785587', 'cos': 2.625},
   'text': {'sm': 'Skilled (Identify the circuit causing an electrical system to fail)',
    'cos': 'Identify the circuit causing an electrical system to fail'}},
  {'id': {'sm': '1149785588', 'cos': 3.4125},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785589', 'cos': 4.2},
   'text': {'sm': 'Expert (Oversee code deb

{'question_id': {'sm': '156451068', 'cos': '2.A.1.c'},
 'page_number': 21,
 'question_number': {'sm': 81, 'cos': 40},
 'question_family': 'single_choice',
 'question_text': {'sm': "How well does your writing effectively communicate to an audience's particular need",
  'cos': "How well does your writing effectively communicate to an audience's particular need?"},
 'answers': [{'id': {'sm': '1149785590', 'cos': 1.624},
   'text': {'sm': 'Beginner (Take a telephone message)',
    'cos': 'Take a telephone message'}},
  {'id': {'sm': '1149785591', 'cos': 2.467},
   'text': {'sm': 'Basic', 'cos': ''}},
  {'id': {'sm': '1149785592', 'cos': 3.31},
   'text': {'sm': 'Skilled (Write a memo to staff outlining new directives)',
    'cos': 'Write a memo to staff outlining new directives'}},
  {'id': {'sm': '1149785593', 'cos': 4.153},
   'text': {'sm': 'Advanced', 'cos': ''}},
  {'id': {'sm': '1149785594', 'cos': 4.996},
   'text': {'sm': 'Expert (Write a technical manual)',
    'cos': 'Write a tec